In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import avar
import seaborn as sns
# %matplotlib notebook
%matplotlib qt

sns.set()
# plt.rcParams['mathtext.fontset'] = 'stix'
# plt.rcParams['font.family'] = 'STIXGeneral'

path = os.getcwd() + "\\crime_data"
csv_files = glob.glob(os.path.join(path, "*.csv"))
df = pd.read_csv(csv_files[0])

C:\Users\Hossein_Haeri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
df


,crime,number,date,day,location,beat,neighborhood,npu,lat,long
0,BURGLARY-NONRES,90010801.0,1/1/2009,39814,1020 DONALD LEE HOLLOWELL PKWY NW,106,Bankhead,K,33.77292,-84.42078
1,BURGLARY-RESIDENCE,83651892.0,1/1/2009,39814,2163 RIDGEDALE RD.,610,Kirkwood,O,33.75298,-84.31680
2,BURGLARY-RESIDENCE,83662023.0,1/1/2009,39814,800 W. MARIETTA ST,103,English Avenue,L,33.78022,-84.41514
3,AGG ASSAULT,83662137.0,1/1/2009,39814,625 JOSEPH E LOWERY,104,Ashview Heights,T,33.74865,-84.41763
4,BURGLARY-RESIDENCE,90010061.0,1/1/2009,39814,501 TUFTON TRL SE,312,Glenrose Heights,Z,33.66477,-84.38430
...,...,...,...,...,...,...,...,...,...,...
270683,AGG ASSAULT,170593064.0,2/28/2017,42794,425 CHAPEL ST SW,507,Castleberry Hill,M,33.75285,-84.40152
270684,LARCENY-NON VEHICLE,170593149.0,2/28/2017,42794,2151 LENOX RD NE,213,Lindridge/Martin Manor,F,33.81356,-84.35129
270685,AGG ASSAULT,170593210.0,2/28/2017,42794,499 NORTHSIDE CIR NW,207,Loring Heights,E,33.79742,-84.40671
270686,AUTO THEFT,170593240.0,2/28/2017,42794,1749 EMERALD AVE SW,404,Westview,T,33.74232,-84.44505


In [2]:
from scipy.spatial import cKDTree as KDTree
def KLdivergence(x, y):
#       """Compute the Kullback-Leibler divergence between two multivariate samples.
#       Parameters
#       ----------
#       x : 2D array (n,d)
#         Samples from distribution P, which typically represents the true
#         distribution.
#       y : 2D array (m,d)
#         Samples from distribution Q, which typically represents the approximate
#         distribution.
#       Returns
#       -------
#       out : float
#         The estimated Kullback-Leibler divergence D(P||Q).
#       References
#       ----------
#       Pérez-Cruz, F. Kullback-Leibler divergence estimation of
#     continuous distributions IEEE International Symposium on Information
#     Theory, 2008.
#       """
    

    # Check the dimensions are consistent
    x = np.atleast_2d(x)
    y = np.atleast_2d(y)

    n,d = x.shape
    m,dy = y.shape

    assert(d == dy)


    # Build a KD tree representation of the samples and find the nearest neighbour
    # of each point in x.
    xtree = KDTree(x)
    ytree = KDTree(y)

    # Get the first two nearest neighbours for x, since the closest one is the
    # sample itself.
    r = xtree.query(x, k=2, eps=.01, p=2)[0][:,1]
    s = ytree.query(x, k=1, eps=.01, p=2)[0]

    # There is a mistake in the paper. In Eq. 14, the right side misses a negative sign
    # on the first term of the right hand side.
    return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))

In [3]:
# for i, row in data.iterrows():

# plt.scatter(data['long'],data['lat'],marker='.',s=5,color=)
fig, axes = plt.subplots(1, 2)
# 39627(last2009) 75397(last2010)    237550(last2015)  266682(last2016)


D = df.loc[39627:75397,:]
sns.scatterplot(x='long', y='lat', data=D, hue='crime', ax=axes[0], s=10, legend=False)
axes[0].set_title('1/1/2010 - 12/31/2010')
axes[0].axis('equal')


D = df.loc[237550:266682,:]
sns.scatterplot(x='long', y='lat', data=D, hue='crime', ax=axes[1], s=10, legend=True,alpha=0)
axes[1].set_title('1/1/2016 - 12/31/2016')
axes[1].axis('equal')




(-84.563504, -84.277416, 33.614915, 33.897065000000005)

In [5]:
# import warnings
# warnings.filterwarnings('ignore')


# data = df.loc[df['crime'] == 'AUTO THEFT'][['lat','long', 'day']]
data = df[['lat','long', 'day']]

kld_list = []
k = 2980
d = 2
while k > 30:
    i = 39814
    k = int(k/d)
    print(k)
    kld = []
    while i+2*k<len(df):
        mask1 = (data['day'] > i) & (data['day'] <= i+k)
        mask2 = (data['day'] > i+k) & (data['day'] <= i+2*k)
        D1 = data.loc[mask1][['lat','long']].to_numpy()
        D2 = data.loc[mask2][['lat','long']].to_numpy()

        if len(D1)>0 and len(D2)>0:
            print(D1,D2)
            kld.append(KLdivergence(D1,D2))
        i = i+k
        
    kld_list.append(kld)


1490
[[ 33.67957 -84.36996]
 [ 33.77487 -84.38975]
 [ 33.76733 -84.41259]
 ...
 [ 33.80083 -84.42084]
 [ 33.64047 -84.43915]
 [ 33.65513 -84.42915]] [[ 33.81477 -84.34908]
 [ 33.78267 -84.40588]
 [ 33.82883 -84.3652 ]
 ...
 [ 33.79742 -84.40671]
 [ 33.74232 -84.44505]
 [ 33.82494 -84.36684]]


<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: divide by zero encountered in true_divide
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: invalid value encountered in true_divide
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: divide by zero encountered in log
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
C:\Users\Hossein_Haeri\Anaconda3\lib\site-packages\numpy\core\_methods.py:47: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


745
[[ 33.67957 -84.36996]
 [ 33.77487 -84.38975]
 [ 33.76733 -84.41259]
 ...
 [ 33.80388 -84.41278]
 [ 33.73298 -84.43712]
 [ 33.72132 -84.41313]] [[ 33.65252 -84.3889 ]
 [ 33.72666 -84.39934]
 [ 33.81005 -84.4277 ]
 ...
 [ 33.80083 -84.42084]
 [ 33.64047 -84.43915]
 [ 33.65513 -84.42915]]


<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: divide by zero encountered in true_divide
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: invalid value encountered in true_divide
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: divide by zero encountered in log
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
C:\Users\Hossein_Haeri\Anaconda3\lib\site-packages\numpy\core\_methods.py:47: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


[[ 33.65252 -84.3889 ]
 [ 33.72666 -84.39934]
 [ 33.81005 -84.4277 ]
 ...
 [ 33.80083 -84.42084]
 [ 33.64047 -84.43915]
 [ 33.65513 -84.42915]] [[ 33.81477 -84.34908]
 [ 33.78267 -84.40588]
 [ 33.82883 -84.3652 ]
 ...
 [ 33.69192 -84.51854]
 [ 33.74951 -84.43943]
 [ 33.74951 -84.43943]]


<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: divide by zero encountered in true_divide
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: invalid value encountered in true_divide
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: divide by zero encountered in log
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
C:\Users\Hossein_Haeri\Anaconda3\lib\site-packages\numpy\core\_methods.py:47: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


[[ 33.81477 -84.34908]
 [ 33.78267 -84.40588]
 [ 33.82883 -84.3652 ]
 ...
 [ 33.69192 -84.51854]
 [ 33.74951 -84.43943]
 [ 33.74951 -84.43943]] [[ 33.81265 -84.42536]
 [ 33.69229 -84.52537]
 [ 33.64072 -84.44435]
 ...
 [ 33.79742 -84.40671]
 [ 33.74232 -84.44505]
 [ 33.82494 -84.36684]]


<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: divide by zero encountered in true_divide
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: invalid value encountered in true_divide
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
<ipython-input-2-d8c2acce2409>:46: RuntimeWarning: divide by zero encountered in log
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))
C:\Users\Hossein_Haeri\Anaconda3\lib\site-packages\numpy\core\_methods.py:47: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


KeyboardInterrupt: 

In [95]:
kld_list

[[nan], [nan, nan, nan]]